<a href="https://colab.research.google.com/github/younghwani/ScalpProject/blob/master/scalpVGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VGG16

### Library

In [ ]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf

import shutil
import glob
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import zipfile

### Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Data file copy (from drive to colab kernel)
original_data_path = '/content/drive/MyDrive/Data/HairLossData.zip'
new_data_path = '/content'

shutil.copy(original_data_path, new_data_path)

In [ ]:
# Unzip
path_to_zip_file = '/content/HairLossData.zip'
directory_to_extract_to = '/content'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
trainDataPath = '/content/HairLossData/Training/image'
validDataPath = '/content/HairLossData/Validation/image'

trData = ImageDataGenerator()
trainData = trData.flow_from_directory(directory=trainDataPath, target_size=(224,224))
valData = ImageDataGenerator()
validData = valData.flow_from_directory(directory=validDataPath, target_size=(224,224))

### Model

In [ ]:
model = Sequential()
# Conv2D * 2 (64 channel)
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
# MaxPool2D (2 * 2)
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Conv2D * 2 (128 channel)
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# MaxPool2D (2 * 2)
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Conv2D * 3 (256 channel)
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# MaxPool2D (2 * 2)
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Conv2D * 3 (512 channel)
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# MaxPool2D (2 * 2)
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Conv2D * 3 (512 channel)
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# MaxPool2D (2 * 2)
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Flatten
model.add(Flatten())
# Dense * 2 (4096)
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
# Dense (output, softmax)
model.add(Dense(units=4, activation="softmax"))

In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt, 
              loss=keras.losses.categorical_crossentropy, 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             period=1)
early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=20, 
                      verbose=1, 
                      mode='auto')
# ImageDataGenerator를 사용하여 데이터를 전달하므로 model.fit_generator를 사용한다.
hist = model.fit_generator(steps_per_epoch=100,
                           generator=trainData, 
                           validation_data= validData, 
                           validation_steps=10,
                           epochs=100,
                           callbacks=[checkpoint,early])

In [ ]:
model.save_weights("vgg16_1.h5")

In [ ]:
shutil.copy("vgg16_1.h5", "/content/drive/MyDrive")

In [ ]:
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

### Predict

In [ ]:
from keras.preprocessing import image
img = image.load_img("image.jpeg",target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
from keras.models import load_model
saved_model = load_model("vgg16_1.h5")
output = saved_model.predict(img)